In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
folder = r'C:\Users\mazhang\Documents\Purchasing\Amazon\2022 Weekly Report'
files = os.listdir(folder)

excel_list = []
for file in files:
    if file.endswith('.xlsx'):
        df = pd.read_excel(os.path.join(folder, file), sheet_name = None)
        excel_list.append(df)

In [ ]:
df = pd.DataFrame()
for excel in range(len(excel_list)):
    
    df_excel = excel_list[excel]['Sheet1']
    print(df_excel.shape)
    df = df.append(df_excel, ignore_index = True)

In [ ]:
df = df.sort_values(by = ['Week'])
df

In [ ]:
pd.set_option('display.precision', 0)

df1 = df[['Week', 'Facility', 'Cost Center', 'Class', 'Commodity', 'Order Quantity']]
df1 = df1.fillna('NA')
df1.columns = df1.columns.str.strip()
df1

In [ ]:
df2 = df1.groupby(['Week', 'Facility', 'Cost Center', 'Class', 'Commodity']).agg({'Order Quantity': 'sum'})
df2

In [ ]:
df3 = df2.pivot_table(columns = ['Week'], index = ['Facility', 'Cost Center', 'Class', 'Commodity'], 
                      values = 'Order Quantity')
df3 = df3.fillna(0)
df3

In [ ]:
week_numbers = [str(x) for x in df3.columns]
df3.columns = ['Week ' + x for x in week_numbers]
df3

In [ ]:
df4 = df3.reset_index()
df4['Total Quantity'] = df4.sum(axis = 1)
df4

In [ ]:
def highlight_mean_greater(s):
    
    s = s.replace(0, np.NaN)
    is_max = s > s.mean()
    
    return ['background-color: lightcoral' if i else '' for i in is_max]

In [ ]:
grouped = df4.groupby(df4['Facility'])

with pd.ExcelWriter('2022 Amazon Order Weekly Count Report.xlsx') as writer:
    
    (df4.style
     .apply(lambda x: ['background-color: lightseagreen' 
                          if ((i >= 0 and i < (len(df3.columns) - 1)) and                               
                              (x.iloc[i] > 0 and (x.iloc[i+1] > 0 or x.iloc[i-1] > 0)))                           
                          else "" for i, v in enumerate(x)], axis = 1, subset = list(df3.columns))
     .apply(highlight_mean_greater, axis = 1, subset = list(df3.columns))
     .to_excel(writer, index = False, header = True, sheet_name = 'Summary'))
    
    for name in df4['Facility'].unique():
        
        df = grouped.get_group(name)
        
        (df.style
         
         .apply(lambda x: ['background-color: lightseagreen' 
                          if ((i >= 0 and i < (len(df3.columns) - 1)) and                               
                              (x.iloc[i] > 0 and (x.iloc[i+1] > 0 or x.iloc[i-1] > 0)))
                           
                          else "" for i, v in enumerate(x)], axis = 1, subset = list(df3.columns))
         
         .apply(highlight_mean_greater, axis = 1, subset = list(df3.columns))
         
         .to_excel(writer, sheet_name = name, index = False))